# 사전 데이터 불러오기 + 합치기

In [1]:
import pandas as pd

df = pd.read_excel('프로젝트/사전/all_keywords.xlsx', index_col=0)

FileNotFoundError: [Errno 2] No such file or directory: '프로젝트/사전/all_keywords.xlsx'

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df = df.set_index('hs_code')

In [ ]:
result_dict = df.to_dict()
flattened_dict = {key: inner_value for inner_dict in result_dict.values() for key, inner_value in inner_dict.items()}

# print(flattened_dict)

# 테스트 데이터 불러오기

In [ ]:
import pandas as pd
data = pd.read_excel('./프로젝트/사전/dict_test_total.xlsx', index_col=0)

In [ ]:
data['hs code'] = data['hs code'].astype(str).str[:4]

In [ ]:
data = data.drop_duplicates()

In [ ]:
import re
# hs 코드 체크 함수
def check_code(code):
    p = re.compile('8[45]\w*')  # 84로 시작하는 패턴
    m = p.match(code)  # hs코드에서 패턴에 맞는 거 찾기 
    
    if m is None:  # 패턴에 맞지 않는 경우(패턴에 맞지 않으면 none 출력)
        return None  # 패턴에 맞지 않으면 na값으로 반환
    
    else:  # 패턴에 맞는 경우
        return m.group()  # 패턴에 맞는 코드(4자리수) 반환

# hs code 열에 hs 코드 체크 함수 적용
data['hs code'] = data['hs code'].apply(lambda x : check_code(x))

In [ ]:
data.dropna(inplace=True)

In [ ]:
data.info()

# 유사도

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
cnt = 0
# HS 코드와 설명서 데이터
hs_code_documents = flattened_dict

# 입력 설명서
for num, input_document in zip(data['hs code'], data['name_des']):

    # HS 코드 문서와 입력 설명서를 하나로 합침
    documents = list(hs_code_documents.values())
    documents.append(input_document)
    # TF-IDF 벡터화 객체 생성
    tfidf = TfidfVectorizer()
    
    # description
    tfidf_matrix = tfidf.fit_transform(documents)
    # 입력 설명서와 각 HS 코드 간의 코사인 유사도 계산
    similarities = cosine_similarity(tfidf_matrix[-1], tfidf_matrix[:-1])
    
    # 유사도가 가장 높은 HS 코드 인덱스
    most_similar_hs_code_index = similarities.argmax()
    
    # 유사도가 가장 높은 HS 코드와 유사도 값
    most_similar_hs_code = list(hs_code_documents.keys())[most_similar_hs_code_index]
    similarity_value = similarities[0][most_similar_hs_code_index]

    
    # print(f'원래 HS 코드: {num}', end=" ")
    # print(f"유사: {most_similar_hs_code}")
    # print(f"유사도: {similarity_value}")
    if int(num) == most_similar_hs_code:
        cnt += 1 
print(cnt)

# 전처리 추가

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import string

# # 필요한 nltk 데이터 다운로드
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('wordnet')
cnt = 0
# HS 코드와 설명서 데이터
hs_code_documents = flattened_dict

# 입력 설명서
for num, input_document in zip(data['hs code'], data['name_des']):

    # 구두점 제거 함수
    def remove_punctuation(text):
        return text.translate(str.maketrans('', '', string.punctuation))
    
    # 불용어 제거 및 단어의 형태소 분석을 위한 객체 생성
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    
    # 입력 설명서 전처리
    input_document = remove_punctuation(input_document)
    input_tokens = word_tokenize(input_document.lower())  # 소문자로 변환 및 토큰화
    input_tokens = [lemmatizer.lemmatize(word) for word in input_tokens if word not in stop_words]
    
    # HS 코드 문서 전처리
    processed_hs_code_documents = {}
    for code, doc in hs_code_documents.items():
        doc = remove_punctuation(doc)
        tokens = word_tokenize(doc.lower())  # 소문자로 변환 및 토큰화
        tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
        processed_hs_code_documents[code] = ' '.join(tokens)
    
    # TF-IDF 벡터화 객체 생성
    tfidf = TfidfVectorizer()
    documents = list(processed_hs_code_documents.values())
    documents.append(' '.join(input_tokens))  # 입력 설명서를 단일 문자열로 추가
    
    # TF-IDF 벡터화
    tfidf_matrix = tfidf.fit_transform(documents)
    
    # 입력 설명서와 각 HS 코드 간의 코사인 유사도 계산
    similarities = cosine_similarity(tfidf_matrix[-1], tfidf_matrix[:-1])
    
    # 유사도가 가장 높은 HS 코드 인덱스
    most_similar_hs_code_index = similarities.argmax()
    
    # 유사도가 가장 높은 HS 코드와 유사도 값
    most_similar_hs_code = list(hs_code_documents.keys())[most_similar_hs_code_index]
    similarity_value = similarities[0][most_similar_hs_code_index]
    
    # print(f'원래 HS 코드: {num}', end=" ")
    # print(f"유사: {most_similar_hs_code}")
    # print(f"유사도: {similarity_value}")
    if int(num) == most_similar_hs_code:
        cnt += 1 
print('맞춘개수: ',cnt)
print('예측 정확도: ', (cnt/len(data['name_des']))*100)

# 가중치 추가

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import string

# # 필요한 nltk 데이터 다운로드
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('wordnet')
cnt = 0
# HS 코드와 설명서 데이터
hs_code_documents = flattened_dict

# 입력 설명서
for num, input_document in zip(data['hs code'], data['name_des']):

    # 구두점 제거 함수
    def remove_punctuation(text):
        return text.translate(str.maketrans('', '', string.punctuation))
    
    # 불용어 제거 및 단어의 형태소 분석을 위한 객체 생성
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    
    # 입력 설명서 전처리
    input_document = remove_punctuation(input_document)
    input_tokens = word_tokenize(input_document.lower())  # 소문자로 변환 및 토큰화
    input_tokens = [lemmatizer.lemmatize(word) for word in input_tokens if word not in stop_words]
    
    # HS 코드 문서 전처리
    processed_hs_code_documents = {}
    for code, doc in hs_code_documents.items():
        doc = remove_punctuation(doc)
        tokens = word_tokenize(doc.lower())  # 소문자로 변환 및 토큰화
        tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
        processed_hs_code_documents[code] = ' '.join(tokens)
    
    # TF-IDF 벡터화 객체 생성
    tfidf = TfidfVectorizer()
    documents = list(processed_hs_code_documents.values())
    tfidf_matrix = tfidf.fit(documents)
    documents.append(' '.join(input_tokens))  # 입력 설명서를 단일 문자열로 추가
    
    # TF-IDF 벡터화
    tfidf_matrix = tfidf.transform(documents)
    # tfidf_matrix[:, tfidf.vocabulary_['solely']] *= 2.0
    # tfidf_matrix[:, tfidf.vocabulary_['principally']] *= 2.0
    # 입력 설명서와 각 HS 코드 간의 코사인 유사도 계산
    similarities = cosine_similarity(tfidf_matrix[-1], tfidf_matrix[:-1])
    
    # 유사도가 가장 높은 HS 코드 인덱스
    most_similar_hs_code_index = similarities.argmax()
    
    # 유사도가 가장 높은 HS 코드와 유사도 값
    most_similar_hs_code = list(hs_code_documents.keys())[most_similar_hs_code_index]
    similarity_value = similarities[0][most_similar_hs_code_index]
    
    # print(f'원래 HS 코드: {num}', end=" ")
    # print(f"유사: {most_similar_hs_code}")
    # print(f"유사도: {similarity_value}")
    if int(num) == most_similar_hs_code:
        cnt += 1 
print(cnt)

# 상위 3개

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import string
import numpy as np

# # 필요한 nltk 데이터 다운로드
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('wordnet')
cnt = 0
# HS 코드와 설명서 데이터
hs_code_documents = flattened_dict

# 입력 설명서
for num, input_document in zip(data['hs code'], data['name_des']):

    # 구두점 제거 함수
    def remove_punctuation(text):
        return text.translate(str.maketrans('', '', string.punctuation))
    
    # 불용어 제거 및 단어의 형태소 분석을 위한 객체 생성
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    
    # 입력 설명서 전처리
    input_document = remove_punctuation(input_document)
    input_tokens = word_tokenize(input_document.lower())  # 소문자로 변환 및 토큰화
    input_tokens = [lemmatizer.lemmatize(word) for word in input_tokens if word not in stop_words]
    
    # HS 코드 문서 전처리
    processed_hs_code_documents = {}
    for code, doc in hs_code_documents.items():
        doc = remove_punctuation(doc)
        tokens = word_tokenize(doc.lower())  # 소문자로 변환 및 토큰화
        tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
        processed_hs_code_documents[code] = ' '.join(tokens)
    
    # TF-IDF 벡터화 객체 생성
    tfidf = TfidfVectorizer()
    documents = list(processed_hs_code_documents.values())
    tfidf_matrix = tfidf.fit(documents)
    documents.append(' '.join(input_tokens))  # 입력 설명서를 단일 문자열로 추가
    
    # TF-IDF 벡터화
    tfidf_matrix = tfidf.transform(documents)

    # 입력 설명서와 각 HS 코드 간의 코사인 유사도 계산
    similarities = cosine_similarity(tfidf_matrix[-1], tfidf_matrix[:-1])

    
    # 유사도가 가장 높은 상위 3개 HS 코드 인덱스 및 값
    most_similar_hs_code_indices = np.argsort(similarities)[0][-3:][::-1]  # 상위 3개 인덱스
    top_3_similarity_values = similarities[0][most_similar_hs_code_indices] # 상위 3개 유사도 값
    top_3_hs_codes = [list(hs_code_documents.keys())[i] for i in most_similar_hs_code_indices]  # 상위 3개 HS 코드
    
    
    # 원래 HS 코드 출력
    # print(f'원래 HS 코드: {num}', end=" ")
    # print(f"상위 3개 유사도 값: {top_3_similarity_values}")
    # print(f"상위 3개 HS 코드: {top_3_hs_codes}")
    
    # 입력 HS 코드와 유사한 상위 3개 HS 코드 중 일치하는 경우 cnt 증가
    if int(num) in top_3_hs_codes:
        cnt += 1

print('맞춘개수: ', cnt)
print('예측 정확도: ', (cnt/len(data['name_des']))*100)